In [1]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time

driver_path = "C:\chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')

browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)
browser.get('https://선한영향력가게.com/store/view?idx=1079')


# 403Error 방지: url접근변수 설정
current_url = browser.current_url
headers = {'User-Agent':'Mozilla/5.0'}
req = Request(current_url, headers = headers)

# http source 추출을 위한 변수설정
html = urlopen(req)
source = html.read().decode("utf-8")
code = html.getcode()

# http 출력로직
if code == 200 :
    print("url 정상 추출 코드 : 200\n")
else :
    print("HTTP-Error")

# html을 dom화 하여서 객체화
soup = BeautifulSoup (source)


# 문서의 제목확인
print ("문서의 제목: ", soup.title.text, type(soup))

# 식당정보가 담겨있는 태그수
stores = soup.select("div.store-desc")
print ("식당 정보 담겨있는 태그 수 : ", len(stores), type(stores))

stores_info=[]

for store in stores:
    store_name = store.select("em.underline")
    store_address = store.select("div.store-desc>dl address")
    store_tel = store.select("div.store-desc>dl a")
    store_div = store.select("div.store-desc>dl address")
    store_runtime = store.select("div.store-desc > dl > div:nth-child(2) > dd > div")
    store_for = store.select("div.store-desc > dl > div:nth-child(3) > dd")
    
    if len(store_name) > 0 :
        stores_info.append({
            "식당 이름" : store_name[0].text,
            "식당 주소" : store_address[0].text,
            "식당 번호" : store_tel[0].text,
            "영업 시간" : store_runtime[0].text.replace("\u200b",""),
            "제공 대상" : store_for[0].text.replace("\r\n","").strip()
        })
print(stores_info, type(stores_info))

for i in range (1, 2):
    url_1="https://선한영향력가게.com/store/view?idx="
    browser.get(url_1+str(i))
    
    try:
        WebDriverWait (browser, 1).until(EC.alert_is_present())
        alert= browser.switch_to.alert
        alert.accept()
        
    except:
        # http source 추출을 위한 변수설정
        html = urlopen(req)
        source = html.read().decode("utf-8")
        code = html.getcode()

        soup = BeautifulSoup (source)

        html = urlopen(browser.current_url)
        soup = BeautifulSoup (html.read().decode("utf-8"))
        stores = soup.select("div.store-desc")


        for store in stores:
            store_name = store.select("em.underline")
            store_address = store.select("div.store-desc dl address")
            store_tel = store.select("div.store-desc>dl a")
            store_div = store.select("div.store-desc>dl address")
            store_runtime = store.select("div.store-desc > dl > div:nth-child(2) > dd > div")
            store_for = store.select("div.store-desc > dl > div:nth-child(3) > dd")

            stores_info.append({
                    "식당 이름" : store_name[0].text,
                    "식당 주소" : store_address[0].text,
                    "식당 번호" : store_tel[0].text if len(store_tel) > 0 else "",
                    "영업 시간" : store_runtime[0].text.replace("\u200b","") if len(store_runtime) > 0 else "",
                    "제공 대상" : store_for[0].text.replace("\r\n","").strip()if len(store_runtime) > 0 else ""
                })
# else browser.switchTo().alert().accept() :  
        
print(stores_info, type(stores_info))

# str1= "오전오후시분"
# stores_info1("영업 시간").replace(str1,"")

stores=pd.DataFrame(stores_info)
stores.to_csv("C:/mini_project/stores.csv", header=False, index=False)
stores

seoul_stores=stores[stores["식당 주소"].str.contains("서울")]
seoul_stores


C:\Users\BIT\AppData\Local\Temp\ipykernel_1060\2819396815.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)
C:\Users\BIT\AppData\Local\Temp\ipykernel_1060\2819396815.py:15: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)


url 정상 추출 코드 : 200

문서의 제목:  선한영향력가게::대연생고깃간 <class 'bs4.BeautifulSoup'>
식당 정보 담겨있는 태그 수 :  1 <class 'bs4.element.ResultSet'>
[{'식당 이름': '대연생고깃간', '식당 주소': '서울 성북구 돌곶이로22나길 29 1층 대연생고깃간', '식당 번호': '010-8475-6567', '영업 시간': '오전9시30분~ 저녁9시30분', '제공 대상': '아이 본인만                                                                                                                                                                                                                                                    결식아동'}] <class 'list'>
[{'식당 이름': '대연생고깃간', '식당 주소': '서울 성북구 돌곶이로22나길 29 1층 대연생고깃간', '식당 번호': '010-8475-6567', '영업 시간': '오전9시30분~ 저녁9시30분', '제공 대상': '아이 본인만                                                                                                                                                                                                                                                    결식아동'}] <class 'list'>


,식당 이름,식당 주소,식당 번호,영업 시간,제공 대상
0,대연생고깃간,서울 성북구 돌곶이로22나길 29 1층 대연생고깃간,010-8475-6567,오전9시30분~ 저녁9시30분,아이 본인만 ...


In [2]:
# 2/28 미니프로젝트 1일차
# 선한영향력 json 데이터 파일받아오기
# 데이터프레임으로 전환후 csv 파일로저장

import json
import requests


def send_api (path, method):
    API_HOST = "https://xn--o39akkz01az4ip7f4xzwoa.com/"
    url = API_HOST + path
    headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
    body = {}

    try:
        if method == 'GET':
            response = requests.get(url, headers=headers)
        elif method == 'POST':
            response = requests.post(url, headers=headers,
                                     data=json.dumps(body, ensure_ascii=False, indent="\t"))
#        print("response status %r" % response.status_code)
#        print("response text %r" % response.text)
    except Exception as ex:
        print(ex)
        
    info = json.loads(response.text)
    store_info=pd.DataFrame(info['storeList'])
    
    return store_info

stores=send_api("/store/search", "POST")
stores.to_csv("C:\/mini_project/stores.csv", index=False)
stores.tail()

,area,item_code,target2_code,addr2,lng,addr1,open_info,target2_etc,type,close_info,zipcode,img_name,name,target1_code,target1_etc,idx,item_etc,lat
1001,경남,99,01,1층 OJY Beauty House,128.897337265659,경남 김해시 김해대로2471번길 8,월~토 오전10~오후8시,,99,매주 일요일,50934,6ad39d97f4a246598a601b0a6092c94b.jpg,O · J Y Beauty House,01,,19,"커트 무료 , 커트제외전시술50%",35.2287008432853
1002,경남,02,"01,02",148호 카페두다,128.684884477157,경남 창원시 성산구 창원대로 888,07:00~21:00,,01,일요일,51532,NaN,카페 두번째다락방 두다,02,,18,,35.2087938761328
1003,인천,99,"01,02",휴먼빌딩 102호 냠냠족발,126.708659384889,인천 남동구 남동대로702번길 25,16시~24시,,01,매주 수요일,21577,c23eb201da984f2584292a85dc145f60.jpg,냠냠족발&보쌈 구월점,01,,17,싱글세트,37.4450764326939
1004,전북,99,"01,02,99",상가 101호,127.111360492773,전북 전주시 완산구 홍산북로 83,09시~18시,소방공무원 50% 할인(공무원증확인),01,"공휴일, 매주 일요일 휴무",54969,c903bb7a61904fa5bbe49277e8debe22.jpg,카페더헤링본,99,아이 본인 및 가족 등 동반인 모두,16,"모든메뉴, 없는메뉴도 만들어드립니다.",35.8183535350013
1005,서울,02,01,1층 EYE OH,126.927395065288,서울 서대문구 신촌로 1,10:00~21:00,,99,없음,03785,NaN,아이오안경원,01,,15,,37.5588739584805


In [3]:
# 3/2 미니프로젝트 3일차
# 데이터 프레임 mysql 에 저장하기

import MySQLdb
from sqlalchemy import create_engine
import pymysql

columns=['area',
         'item_code',
         'target2_code',
         'addr2',
         'lng',
         'addr1',
         'open_info',
         'target2_etc',
         'type',
         'close_info',
         'zipcode',
         'img_name',
         'name',
         'target1_code',
         'target1_etc',
         'idx',
         'item_etc',
         'lat'
]

df= pd.DataFrame(stores, columns=columns)


host = "127.0.0.1"
user = "root"
password = "bit123"
database = "mini_project"

engine = create_engine(f"mysql+mysqldb://{user}:{password}"\
                    f"@{host}:3306/{database}",
                    encoding="utf-8")
conn = engine.connect()

df.to_sql(name="stores_db",
             con=engine,
             if_exists="append",
             index=False)

conn.close()